<a href="https://colab.research.google.com/github/celinagacias/ph-icd-library/blob/master/01%20Prepare%20Files/a%20-%20Move%20Files%20to%20BQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reading Files into BQ

1. Claims in .csv
2. ACR medical and procedure codes
3. TDRG rules
4. RVS-ICD9 mapping from 2012



### Claims
Steps for loading big .csv files from GCS to BQ from: 
- https://medium.com/@philipplies/transferring-data-from-google-drive-to-google-cloud-storage-using-google-colab-96e088a8c041

In [0]:
!pip install -U -q PyDrive

import pandas as pd
import pandas_gbq as gbq
import numpy as np
import xlrd
import itertools
import gspread

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, drive
from oauth2client.client import GoogleCredentials

In [0]:
#Authenticate user
auth.authenticate_user()
gauth = GoogleAuth()

#Mount GDrive
gdrive = GoogleDrive(gauth)
gauth.credentials = GoogleCredentials.get_application_default()
drive.mount('/content/drive')
gc = gspread.authorize(GoogleCredentials.get_application_default())

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### 1. Claims

In [0]:
#Connect to the GCS bucket
project_id = 'drg-claims'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://drg_2020/


In [0]:
#Copy files from GDrive to the GCS bucket
bucket = 'drg_2020'
!gsutil -m cp -r /content/drive/My\ Drive/PPM/01_DRG/DRG_Classification/0_Major_Datasets/claims_* gs://drg_2020

Copying file:///content/drive/My Drive/PPM/01_DRG/DRG_Classification/0_Major_Datasets/claims_2016.csv [Content-Type=text/csv]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file:///content/drive/My Drive/PPM/01_DRG/DRG_Classification/0_Major_Datasets/claims_2018.csv [Content-Type=text/csv]...
Copying file:///content/drive/My Drive/PPM/01_DRG/DRG_Classificati

In [0]:
#Specify the schema and load into BQ
from google.cloud import bigquery
client = bigquery.Client('drg-claims')
dataset_id = 'claims'

  #Set job_config
dataset_ref = client.dataset(dataset_id)
job_config = bigquery.LoadJobConfig(
  write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE,
  schema = [
    bigquery.SchemaField("series", "STRING"),
    bigquery.SchemaField("check_year", "FLOAT64"),
    bigquery.SchemaField("pseudo_pin", "STRING"),
    bigquery.SchemaField("patient_type", "STRING"),
    bigquery.SchemaField("member_category", "STRING"),
    bigquery.SchemaField("member_subcategory", "STRING"),
    bigquery.SchemaField("patage", "FLOAT64"),
    bigquery.SchemaField("patsex", "STRING"),
    bigquery.SchemaField("date_adm", "STRING"),
    bigquery.SchemaField("date_dis", "STRING"),
    bigquery.SchemaField("disposition", "STRING"),
    bigquery.SchemaField("status", "STRING"),
    bigquery.SchemaField("accommodation_type", "STRING"),
    bigquery.SchemaField("inst_code", "STRING"),
    bigquery.SchemaField("out_patient", "STRING"),
    bigquery.SchemaField("c1", "STRING"),
    bigquery.SchemaField("c2", "STRING"),
    bigquery.SchemaField("icdcode1", "STRING"),
    bigquery.SchemaField("icdcode2", "STRING"),
    bigquery.SchemaField("icdcode3", "STRING"),
    bigquery.SchemaField("icdcode4", "STRING"),
    bigquery.SchemaField("icdcode5", "STRING"),
    bigquery.SchemaField("icdcode6", "STRING"),
    bigquery.SchemaField("icdcode7", "STRING"),
    bigquery.SchemaField("icdcode8", "STRING"),
    bigquery.SchemaField("icdcode9", "STRING"),
    bigquery.SchemaField("icdcode10", "STRING"),
    bigquery.SchemaField("rvscode1", "STRING"),
    bigquery.SchemaField("rvscode2", "STRING"),
    bigquery.SchemaField("rvscode3", "STRING"),
    bigquery.SchemaField("rvscode4", "STRING"),
    bigquery.SchemaField("rvscode5", "STRING"),
    bigquery.SchemaField("claims_amount", "FLOAT64"),
    bigquery.SchemaField("actual_amount", "FLOAT64"),
    bigquery.SchemaField("inst_name", "STRING"),
    bigquery.SchemaField("cat", "STRING"),
    bigquery.SchemaField("municipality_name", "STRING"),
    bigquery.SchemaField("province_name", "STRING"),
    bigquery.SchemaField("soc_sector", "STRING")
    ],
  skip_leading_rows = 1,
  ignore_unknown_values = True,
  max_bad_records = 1000000) #Not ideal!

#Load the tables from GCS

for claims in ["claims_2016", "claims_2017", "claims_2018"]:
  uri = "gs://drg_2020/" + claims + ".csv"
  load_job = client.load_table_from_uri(
      uri, dataset_ref.table(claims), job_config=job_config
  ) 

  print("Starting job {} for {}".format(load_job.job_id, claims))

  load_job.result()
  print("Job finished.")

  #Throw the table into BQ
  destination_table = client.get_table(dataset_ref.table(claims))
  print("Loaded {} rows.".format(destination_table.num_rows))

Starting job 65345de9-f10b-4232-b96b-5454fd76de31 for claims_2017
Job finished.
Loaded 11131960 rows.
Starting job 3c0c4e9c-cab6-4718-959f-6b4a24453265 for claims_2018
Job finished.
Loaded 11761376 rows.


In [0]:
#Created partitioned tables, fix some columns
for claims in ["claims_2016", "claims_2017", "claims_2018"]:
  table_id = "drg-claims.claims.{}".format(claims)
  job_config_new = bigquery.QueryJobConfig()

  partition = """
  CREATE TABLE {}
  PARTITION BY date_adm
  AS
  SELECT
    * EXCEPT(check_year,
      date_adm,
      date_dis),
    CAST(check_year AS INT64) check_year,
    DATE(PARSE_DATETIME("%m/%d/%Y %I:%M %p",
        date_adm)) date_adm,
    DATE(PARSE_DATETIME("%m/%d/%Y %I:%M %p",
        date_dis)) date_dis,
    PARSE_DATETIME("%m/%d/%Y %I:%M %p",
      date_adm) date_adm_time,
    PARSE_DATETIME("%m/%d/%Y %I:%M %p",
      date_dis) date_dis_time
  FROM
    `{}`
  """.format(table_id + "_partitioned", table_id)

  query_job = client.query(partition, job_config=job_config_new)  # Make an API request.
  query_job.result()  # Wait for the job to complete.

  print("Partitioned table loaded to the table {}".format(table_id))

Partitioned table loaded to the table drg-claims.claims.claims_2017
Partitioned table loaded to the table drg-claims.claims.claims_2018


### ACR Codes

In [0]:
#List down all the files in the "Major datasets" GFolder
major_id = "1JGNQUf07w0tvLoxsw-5-uhe7vUHxpLIb"
major_files = gdrive.ListFile({'q': "'" + major_id + "'" + " in parents and trashed=false"}).GetList()

#Obtain the files for medical and procedural case rates
acr_files = [f for f in major_files if 'ACR' in f['title']]
for a in acr_files:
  acr_gfile = gdrive.CreateFile({'id': a['id']})
  acr_gfile.GetContentFile(a['title'])

  #Read in medical ACR
acr_diag = pd.ExcelFile('ACR Medical Case Rates.xls').parse('medical')
acr_diag = acr_diag.iloc[4:, :6]
acr_diag.columns = ['icd10','icd_desc','acr_group','cr','pf','hci']

  #Read in procedural ACR
acr_prod = pd.ExcelFile('ACR Procedure Case Rate.xlsx').parse('ANNEX 2 REVISION 3')
acr_prod = acr_prod.iloc[4:]
acr_prod.columns= ['rvs','rvu','rvs_desc','cr','pf','hci']
acr_prod = acr_prod[acr_prod.rvs.notnull()]

#Read them to BQ
gbq.to_gbq(acr_diag, 'acr.diagnoses', project_id='drg-claims', if_exists='replace')
gbq.to_gbq(acr_prod, 'acr.procedures', project_id='drg-claims', if_exists='replace')


1it [00:02,  2.73s/it]

1it [00:04,  4.35s/it]


### TDRG Rules

In [0]:
#Folder with the TDRG rules files
rules_id = "1M0jssvtjMcvR1uVxOw9woKP2jDYlQyAD"
rules_files = gdrive.ListFile({'q': "'" + rules_id + "'" + " in parents and trashed=false"}).GetList()

  #Read them from GDrive and load to BQ
tdrg_files = {'ICD-10 Codes for Reimbursement.xlsx': 'tdrg.icd10',
              'ICD-9-CM Procedures per MDC.xlsx': 'tdrg.icd9_to_mdc',
              'Dagger and Asterisk Codes.xlsx': 'tdrg.dagger_asterisk',
              'ICD-9-CM Procedures.xlsx': 'tdrg.icd9cm',
              'PDC.xlsx': 'tdrg.pdc',
              'DC.xlsx': 'tdrg.dc',
              'CCLM.xlsx': 'tdrg.cclm',
              'ICD-10 with Descriptions.xlsx': 'tdrg.icd10_vx',
              'ICD-9-CM with Descriptions.xlsx': 'tdrg.icd9_vx'}

for filename in ['ICD-9-CM with Descriptions.xlsx']:
  efile = [f for f in rules_files if f['title'] == filename][0]
  gfile = gdrive.CreateFile({'id': efile['id']})
  gfile.GetContentFile(gfile['title'])
  excel = pd.ExcelFile(gfile['title']).parse(0)
  excel = excel[[c for c in excel.columns.values if "Unnamed:" not in c]]
  gbq.to_gbq(excel, tdrg_files[filename], project_id='drg-claims', if_exists='replace')

1it [00:03,  3.39s/it]


### RVS to ICD-9-CM Mapping

In [0]:
#Get all the Excel files in the mapping folder
  #List down the files
# old_map_folder = "1GRjjFOhWMfHnw_MlKdtwTsq26Wl3Zo6W" #The old mapping, raw
map_folder = "1sOji5u3s0aNczXP9FIKRAKyx3ekzi51V" #Old mapping with Doc Marc's comments
map_files = gdrive.ListFile({'q': "'" + map_folder + "'" + " in parents and trashed=false"}).GetList()
map_files = [m for m in map_files if m['mimeType'] == 'application/vnd.ms-excel']
print("There are {} files to compile.".format(len(map_files)))

  #Read all of them as Excel objects into a list
map_excels = []
map_names = []
for f in map_files:
  file_id = f['id']
  file_name = f['title']

  gfile = gdrive.CreateFile({'id': file_id})
  gfile.GetContentFile(file_name)

  map_excels.append(pd.ExcelFile(file_name))
  map_names.append(file_name)

There are 32 files to compile.


In [0]:
#Parse each of the files
organ_filename = {}
mapping_all = []

for organ_file in map_names:
  mapping = pd.ExcelFile(organ_file).parse(0)
  mapping = mapping.iloc[:,:6]

  #Get the organ system
  if organ_file == '26 ICD9CM Other Medicine Services (Radiology).xls':
    organ_system = "OTHER MEDICAL SERVICES"
  else:
    organ_ind = mapping.loc[(mapping.iloc[:,0].notnull()) & (mapping.iloc[:,0].str.lower().str.startswith("organ system"))].index[0]
    organ_system = mapping.iloc[organ_ind, 0].upper().split("ORGAN SYSTEM:")[1].strip()
  
  #Get the actual rows
  rvs_ind = mapping.loc[(mapping.iloc[:,0].notnull()) & (mapping.iloc[:,0].str.lower().str.startswith('rvs code'))].index[0]
  mapping = mapping[rvs_ind+1:]

  #Organize the columns
  if mapping.shape[1] == 6:
    mapping.columns = ['rvs', 'rvs_desc','icd9cm','icd9cm_desc','remarks','feedback']
  else:
    mapping.columns = ['rvs', 'rvs_desc','icd9cm','icd9cm_desc','remarks']
    mapping['feedback'] = np.nan
  mapping['organ_system'] = organ_system
  organ_filename[organ_system] = organ_file

  #Forward fill the empty codes
  mapping['rvs'] = mapping['rvs'].apply(lambda x: x.split('\nAND')[0] if str(x) != 'nan' and '\nAND' in str(x) else str(x))
  mapping[['rvs','rvs_desc']] = mapping[['rvs','rvs_desc']].mask(mapping[['rvs','rvs_desc']]=='nan', None).ffill()
  
  mapping_all.append(mapping)
  print(organ_file, organ_system)

#Put everything together
mapping_all = pd.concat(mapping_all)

32 ICD9CM Musculoskeletal System (Humerus to Wrist).xls MUSCULOSKELETAL (HUMERUS (UPPER ARM) AND ELBOW, FOREARM AND WRIST)
31 ICD9CM Medicine Services.xls MEDICINE SERVICES (90935 - 99360)
30 ICD9CM Urinary System.xls URINARY SYSTEM
29 ICD9CM Skin.xls INTEGUMENTARY SYSTEM - SKIN, SUBCUTANEOUS, AND ACCESSORY STRUCTURES
28 ICD9CM Respiratory System.xls RESPIRATORY SYSTEM
27 ICD9CM Pathology Services.xls PATHOLOGY
26 ICD9CM Other Medicine Services (Radiology).xls OTHER MEDICAL SERVICES
25 ICD9CM Nervous System (Skull to Spinal Cord).xls NERVOUS SYSTEM (SKULL, MENINGES, BRAIN, SPINE AND SPINAL CORD)
24 ICD9CM Musculoskeletal System (Pelvis and Hip).xls MUSCULOSKELETAL - PELVIS AND HIP JOINT
23 ICD9CM Musculoskeletal System (Hands and Fingers).xls MUSCULOSKELETAL -HANDS AND FINGERS
22 ICD9CM Musculoskeletal System (General).xls MUSCULOSKELETAL (GENERAL, HEAD, NECK AND THORAX)
21 ICD9CM Musculoskeletal System (Femur and Legs).xls MUSCULOSKELETAL (FEMUR AND KNEE JOINT, LEG AND ANKLE JOINT)
20

In [0]:
#Load to BQ
gbq.to_gbq(mapping_all,'tdrg.icd9_mapping_2012_feedback', project_id='drg-claims', if_exists='replace')

1it [00:03,  3.55s/it]


# Separate processing for RVS to ICD9 Mapping

In [0]:
#Categorize each RVS code as being: one to one, one to many, or many to one
onetoone = mapping_all[mapping_all.rvs.isin([x for x in mapping_all.rvs if list(mapping_all.rvs).count(x) == 1])].rvs.tolist()
onetomany = mapping_all[mapping_all.rvs.isin([x for x in mapping_all.rvs if list(mapping_all.rvs).count(x) > 1])].rvs.tolist()

mapping_all.loc[mapping_all.rvs.isin(onetoone), 'mapping_status'] = 'one is to one'
mapping_all.loc[mapping_all.rvs.isin(onetomany), 'mapping_status'] = 'one is to many'

In [0]:
#Obtain subsets for each organ system, sort, then output to excel file
# !pip install xlsxwriter
import xlsxwriter

for o in organ_filename:
  sub = mapping_all[mapping_all.organ_system == o]
  sub.sort_values(['mapping_status', 'rvs'], inplace=True)
  sub.rename(columns={'rvs': 'RVS', 'rvs_desc': 'RVS Description', 'icd9cm': 'ICD-9-CM', 
                    'icd9cm_desc': 'ICD-9-CM Description', 'remarks': 'Remarks', 
                    'feedback': 'PhilHealth Feedback', 'organ_system': 'Organ System',
                    'mapping_status': 'Mapping Status'}, inplace=True)

  filename = organ_filename[o] + "x"
  writer = pd.ExcelWriter(filename, engine='xlsxwriter')
  sub.to_excel(writer, sheet_name='Sheet1', index=False)

  workbook = writer.book
  worksheet = writer.sheets['Sheet1']
  format = workbook.add_format({'text_wrap': True})

  # Setting the format but not setting the column width
  worksheet.set_column("B:F", 50, format)
  writer.save()

  #Copy to GDrive folder
  !cp "$filename" /content/drive/"My Drive"/PPM/01_DRG/DRG_Classification/"1 RVS-ICD9CM Mapping"/Datasets/"2.1 Old RVS-ICD Mapping, Sorted"
  print(o)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


MUSCULOSKELETAL (HUMERUS (UPPER ARM) AND ELBOW, FOREARM AND WRIST)
MEDICINE SERVICES (90935 - 99360)
URINARY SYSTEM
INTEGUMENTARY SYSTEM - SKIN, SUBCUTANEOUS, AND ACCESSORY STRUCTURES
RESPIRATORY SYSTEM
PATHOLOGY
OTHER MEDICAL SERVICES
NERVOUS SYSTEM (SKULL, MENINGES, BRAIN, SPINE AND SPINAL CORD)
MUSCULOSKELETAL - PELVIS AND HIP JOINT
MUSCULOSKELETAL -HANDS AND FINGERS
MUSCULOSKELETAL (GENERAL, HEAD, NECK AND THORAX)
MUSCULOSKELETAL (FEMUR AND KNEE JOINT, LEG AND ANKLE JOINT)
MUSCULOSKELETAL (FOOT AND TOES, BODY AND UPPER EXTREMITY, LOWER EXTREMITY)
MUSCULOSKELETAL - ABDOMEN AND SHOULDER
MEDIASTINUM AND DIAPHRAGM
MATERNITY CARE AND DELIVERY
MALE GENITAL SYSTEM
HEMIC AND LYMPHATIC SYSTEM (SPLEEN, LYMPHNODES, LYMPHATIC CHANNELS)
FEMALE GENTIAL SYSTEM
EYE AND OCULAR ADNEXAE
NERVOUS SYSTEM - (EXTRACRANIAL NERVES, PERIPHERAL NERVES, AND AUTONOMIC NERVOUS SYSTEM)
ENDOCRINE SYSTEM
DIGESTIVE SYSTEM - LIPS, VESTIBULE OF MOUTH, TONGUE AND FLOOR OF MOUTH, DENTOALVEOLAR, PALATE AND UVULA)
DIGESTI